<a href="https://colab.research.google.com/github/Addykohli/Simple_Polynomial_classifier/blob/main/Classifier_01_poly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Generate pseudo-data for polynomials
# a, b, c are randomly generated from a uniform distribution
# Labels are determined based on the discriminant D = b^2 -4ac

def generate_data(num_samples=5000):
    a = np.random.uniform(-10, 10, num_samples)
    b = np.random.uniform(-10, 10, num_samples)
    c = np.random.uniform(-10, 10, num_samples)

    discriminant = b**2 - 4*a*c
    labels = np.where(discriminant > 0, 2, np.where(discriminant == 0, 1, 0))

    data = np.column_stack((a, b, c))
    return data.astype(np.float32), labels.astype(np.int64)

# Create dataset
N = 5000
X, y = generate_data(N)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train), torch.tensor(X_test)
y_train, y_test = torch.tensor(y_train), torch.tensor(y_test)

# Define the neural network
class PolyClassifier(nn.Module):
    def __init__(self):
        super(PolyClassifier, self).__init__()
        self.fc1 = nn.Linear(3, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 3)  # 3 output classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Model, loss, optimizer
model = PolyClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 90
batch_size = 64

for epoch in range(epochs):
    permutation = torch.randperm(X_train.size(0))
    epoch_loss = 0
    correct = 0
    total = 0

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {correct/total:.4f}")

# Testing
with torch.no_grad():
    test_outputs = model(X_test)
    _, test_predicted = torch.max(test_outputs, 1)
    test_accuracy = (test_predicted == y_test).sum().item() / y_test.size(0)
    print(f"Test Accuracy: {test_accuracy:.4f}")


Epoch 1/90, Loss: 58.7999, Accuracy: 0.5797
Epoch 2/90, Loss: 40.8563, Accuracy: 0.6915
Epoch 3/90, Loss: 31.9947, Accuracy: 0.7552
Epoch 4/90, Loss: 26.5567, Accuracy: 0.7893
Epoch 5/90, Loss: 21.2030, Accuracy: 0.8742
Epoch 6/90, Loss: 15.7561, Accuracy: 0.9200
Epoch 7/90, Loss: 11.9864, Accuracy: 0.9425
Epoch 8/90, Loss: 9.6433, Accuracy: 0.9547
Epoch 9/90, Loss: 8.0615, Accuracy: 0.9607
Epoch 10/90, Loss: 6.9626, Accuracy: 0.9685
Epoch 11/90, Loss: 6.0921, Accuracy: 0.9758
Epoch 12/90, Loss: 5.4645, Accuracy: 0.9792
Epoch 13/90, Loss: 5.0180, Accuracy: 0.9810
Epoch 14/90, Loss: 4.5765, Accuracy: 0.9852
Epoch 15/90, Loss: 4.3029, Accuracy: 0.9845
Epoch 16/90, Loss: 4.0164, Accuracy: 0.9862
Epoch 17/90, Loss: 3.7734, Accuracy: 0.9860
Epoch 18/90, Loss: 3.5579, Accuracy: 0.9870
Epoch 19/90, Loss: 3.3499, Accuracy: 0.9890
Epoch 20/90, Loss: 3.2011, Accuracy: 0.9900
Epoch 21/90, Loss: 3.0916, Accuracy: 0.9888
Epoch 22/90, Loss: 2.9435, Accuracy: 0.9895
Epoch 23/90, Loss: 2.8122, Accurac

In [ ]:
# Example polynomial prediction
try_a, try_b, try_c = 5.0, 0, 0  # Example values
test_input = torch.tensor([[try_a, try_b, try_c]], dtype=torch.float32)
with torch.no_grad():
    prediction = model(test_input)
    _, predicted_class = torch.max(prediction, 1)
    print(f"Predicted class for (a={try_a}, b={try_b}, c={try_c}): {predicted_class.item()}")


Predicted class for (a=5.0, b=0, c=0): 0


In [ ]:
# Generate 100 samples for each class and compute accuracy separately

def generate_fixed_data(class_label, num_samples=100):
    if class_label == 0:
        a = np.random.uniform(1, 10, num_samples)
        b = np.random.uniform(-10, 10, num_samples)
        c = np.random.uniform(1, 10, num_samples)
    elif class_label == 1:
        a = np.random.uniform(1, 10, num_samples)
        b = np.random.uniform(-10, 10, num_samples)
        c = (b ** 2) / (4 * a)
    else:
        a = np.random.uniform(1, 10, num_samples)
        b = np.random.uniform(-10, 10, num_samples)
        c = np.random.uniform(-10, -1, num_samples)

    data = np.column_stack((a, b, c))
    return torch.tensor(data, dtype=torch.float32), torch.full((num_samples,), class_label, dtype=torch.int64)

for class_label in range(3):
    test_data, true_labels = generate_fixed_data(class_label)
    with torch.no_grad():
        predictions = model(test_data)
        _, predicted_classes = torch.max(predictions, 1)
    accuracy = (predicted_classes == true_labels).sum().item() / true_labels.size(0)
    print(f"Accuracy for class {class_label}: {accuracy:.4f}")

Accuracy for class 0: 0.9200
Accuracy for class 1: 0.0000
Accuracy for class 2: 1.0000


**Analysis:**

Random generated data lacks instances of the 1 root class, therefore totally lacks any accuracy. The probability of geting polynomials with 1 root is rare in random coefficients.

This is a case where due to the trainig and testing data were similar, the accuracy seemed to be very high but was lacking the ability to predict an entire class


**Solution**

Generating data to ensure rare instances of 1 root polynomials

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

# Generate balanced data with 2000 samples per class
def generate_balanced_data_v2(samples_per_class=2000):
    data_v2, labels_v2 = [], []

    while len(data_v2) < samples_per_class:
        a_v2, b_v2, c_v2 = np.random.uniform(-10, 10, 3)
        discriminant_v2 = b_v2**2 - 4*a_v2*c_v2
        if discriminant_v2 < 0:
            data_v2.append([a_v2, b_v2, c_v2])
            labels_v2.append(0)


    while len(data_v2) < 2 * samples_per_class:
      a_v2, b_v2, c_v2 = np.random.randint(-10, 10, 3)
      a_v2, b_v2, c_v2 = float(a_v2), float(b_v2), float(c_v2)
      discriminant_v2 = b_v2**2 - 4*a_v2*c_v2
      if discriminant_v2 == 0:
        data_v2.append([a_v2, b_v2, c_v2])
        labels_v2.append(1)

    while len(data_v2) < 3 * samples_per_class:
        a_v2, b_v2, c_v2 = np.random.uniform(-10, 10, 3)
        discriminant_v2 = b_v2**2 - 4*a_v2*c_v2
        if discriminant_v2 > 0:
            data_v2.append([a_v2, b_v2, c_v2])
            labels_v2.append(2)

    data_v2, labels_v2 = np.array(data_v2, dtype=np.float32), np.array(labels_v2, dtype=np.int64)
    indices_v2 = np.random.permutation(len(data_v2))
    return data_v2[indices_v2], labels_v2[indices_v2]

# Create dataset
X_v2, y_v2 = generate_balanced_data_v2(2000)
X_train_v2, X_test_v2, y_train_v2, y_test_v2 = train_test_split(X_v2, y_v2, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_v2, X_test_v2 = torch.tensor(X_train_v2), torch.tensor(X_test_v2)
y_train_v2, y_test_v2 = torch.tensor(y_train_v2), torch.tensor(y_test_v2)

# Define the neural network
class PolyClassifierV2(nn.Module):
    def __init__(self):
        super(PolyClassifierV2, self).__init__()
        self.fc1_v2 = nn.Linear(3, 16)
        self.fc2_v2 = nn.Linear(16, 8)
        self.fc3_v2 = nn.Linear(8, 3)  # 3 output classes
        self.relu_v2 = nn.ReLU()

    def forward(self, x):
        x = self.relu_v2(self.fc1_v2(x))
        x = self.relu_v2(self.fc2_v2(x))
        x = self.fc3_v2(x)
        return x

# Model, loss, optimizer
model_v2 = PolyClassifierV2()
criterion_v2 = nn.CrossEntropyLoss()
optimizer_v2 = optim.Adam(model_v2.parameters(), lr=0.001)

# Training loop
epochs_v2 = 90
batch_size_v2 = 64

for epoch_v2 in range(epochs_v2):
    permutation_v2 = torch.randperm(X_train_v2.size(0))
    epoch_loss_v2 = 0
    correct_v2 = 0
    total_v2 = 0

    for i in range(0, X_train_v2.size(0), batch_size_v2):
        indices_v2 = permutation_v2[i:i+batch_size_v2]
        batch_x_v2, batch_y_v2 = X_train_v2[indices_v2], y_train_v2[indices_v2]

        optimizer_v2.zero_grad()
        outputs_v2 = model_v2(batch_x_v2)
        loss_v2 = criterion_v2(outputs_v2, batch_y_v2)
        loss_v2.backward()
        optimizer_v2.step()

        epoch_loss_v2 += loss_v2.item()
        _, predicted_v2 = torch.max(outputs_v2, 1)
        total_v2 += batch_y_v2.size(0)
        correct_v2 += (predicted_v2 == batch_y_v2).sum().item()

    print(f"Epoch {epoch_v2+1}/{epochs_v2}, Loss: {epoch_loss_v2:.4f}, Accuracy: {correct_v2/total_v2:.4f}")

# Testing
with torch.no_grad():
    test_outputs_v2 = model_v2(X_test_v2)
    _, test_predicted_v2 = torch.max(test_outputs_v2, 1)
    test_accuracy_v2 = (test_predicted_v2 == y_test_v2).sum().item() / y_test_v2.size(0)
    print(f"Test Accuracy: {test_accuracy_v2:.4f}")



Epoch 1/90, Loss: 86.4292, Accuracy: 0.3837
Epoch 2/90, Loss: 75.2253, Accuracy: 0.4842
Epoch 3/90, Loss: 63.5846, Accuracy: 0.6083
Epoch 4/90, Loss: 50.6124, Accuracy: 0.7473
Epoch 5/90, Loss: 41.1722, Accuracy: 0.8333
Epoch 6/90, Loss: 34.9639, Accuracy: 0.8812
Epoch 7/90, Loss: 30.6224, Accuracy: 0.8979
Epoch 8/90, Loss: 27.3851, Accuracy: 0.9033
Epoch 9/90, Loss: 24.9358, Accuracy: 0.9121
Epoch 10/90, Loss: 22.9396, Accuracy: 0.9125
Epoch 11/90, Loss: 21.3061, Accuracy: 0.9167
Epoch 12/90, Loss: 20.0679, Accuracy: 0.9231
Epoch 13/90, Loss: 18.9499, Accuracy: 0.9263
Epoch 14/90, Loss: 18.0271, Accuracy: 0.9256
Epoch 15/90, Loss: 17.1576, Accuracy: 0.9304
Epoch 16/90, Loss: 16.6030, Accuracy: 0.9313
Epoch 17/90, Loss: 15.9862, Accuracy: 0.9331
Epoch 18/90, Loss: 15.3102, Accuracy: 0.9350
Epoch 19/90, Loss: 14.8357, Accuracy: 0.9392
Epoch 20/90, Loss: 14.3777, Accuracy: 0.9413
Epoch 21/90, Loss: 13.9216, Accuracy: 0.9429
Epoch 22/90, Loss: 13.7520, Accuracy: 0.9402
Epoch 23/90, Loss: 

In [ ]:
# Generate 100 samples for each class and compute accuracy separately

def generate_fixed_data(class_label, num_samples=100):
    if class_label == 0:
        a = np.random.uniform(1, 10, num_samples)
        b = np.random.uniform(-10, 10, num_samples)
        c = np.random.uniform(1, 10, num_samples)
    elif class_label == 1:
        a = np.random.uniform(1, 10, num_samples)
        b = np.random.uniform(-10, 10, num_samples)
        c = (b ** 2) / (4 * a)
    else:
        a = np.random.uniform(1, 10, num_samples)
        b = np.random.uniform(-10, 10, num_samples)
        c = np.random.uniform(-10, -1, num_samples)

    data = np.column_stack((a, b, c))
    return torch.tensor(data, dtype=torch.float32), torch.full((num_samples,), class_label, dtype=torch.int64)

for class_label in range(3):
    test_data, true_labels = generate_fixed_data(class_label)
    with torch.no_grad():
        predictions = model_v2(test_data)
        _, predicted_classes = torch.max(predictions, 1)
    accuracy = (predicted_classes == true_labels).sum().item() / true_labels.size(0)
    print(f"Accuracy for class {class_label}: {accuracy:.4f}")

Accuracy for class 0: 0.8400
Accuracy for class 1: 1.0000
Accuracy for class 2: 1.0000


The accuracy is improved significantly